# 1. Data Import and Preprocessing

## 1.1 引入数据集，生成dataframe

In [2]:
import pandas as pd
import numpy as np

data_path ='./dataset_new.csv'
columns_name = ('date', 'time', 'sensor_id', 'value', 'activity', 'status') # status是begin或end
df = pd.read_csv(data_path, names = columns_name)

In [3]:
df.dtypes

date         object
time         object
sensor_id    object
value        object
activity     object
status       object
dtype: object

In [4]:
'''
#一些测试
df.iloc[:,:4]
df_first4Attr = df.iloc[:,:4]
df[df_first4Attr.isnull().values==True]
'''

'\n#一些测试\ndf.iloc[:,:4]\ndf_first4Attr = df.iloc[:,:4]\ndf[df_first4Attr.isnull().values==True]\n'

In [10]:
df['sensor_id'].unique()

array(['M35', 'AD1-A', 'M34', 'M32', 'M33', 'M36', 'M30', 'M29', 'M37',
       'M38', 'M39', 'M41', 'M40', 'M49', 'M46', 'M50', 'M44', 'M47',
       'M48', 'M43', 'M28', 'M42', 'M27', 'M26', 'M25', 'M24', 'M21',
       'M19', 'M18', 'M17', 'M16', 'M15', 'D09', 'M14', 'M13', 'M08',
       'M07', 'M02', 'M45', 'M51', 'D08', 'AD1-B', 'AD1-C', 'D10', 'I03',
       'M06', 'M09', 'M10', 'M01', 'D12', 'L04', 'M23', 'M31', 'D15',
       'D07', 'D14', 'M11', 'M05', 'M03', 'M04', 'M22', 'M12', 'M20',
       'L11', 'D05', 'D03', 'L10', 'L06', 'L13', 'L12', 'L09'],
      dtype=object)

## 1.2 判断缺失值

In [5]:
df.dropna(axis=0, how='all', inplace=True) ## 删去了两行空白行
df

,date,time,sensor_id,value,activity,status
0,02/02/2009,07:15:17,M35,ON,R1_Bed_to_Toilet,begin
1,02/02/2009,07:15:21,AD1-A,2.82231,NaN,NaN
2,02/02/2009,07:15:23,M35,OFF,NaN,NaN
3,02/02/2009,07:15:23,M35,ON,NaN,NaN
4,02/02/2009,07:15:28,AD1-A,2.79926,NaN,NaN
...,...,...,...,...,...,...
138623,04/04/2009,07:26:16,M48,OFF,NaN,NaN
138624,04/04/2009,07:27:47,M48,ON,NaN,NaN
138625,04/04/2009,07:27:50,M47,ON,NaN,NaN
138626,04/04/2009,07:27:59,M47,OFF,NaN,NaN


## 1.3 合并date和time

In [6]:
# df.drop(df.tail(1).index, inplace = True) #从尾部去掉 1 行，这一行是异常行

In [7]:
# df["datetime"] = pd.to_datetime(df["date"] + " "+ df["time"], format='%d/%m/%Y %H:%M:%S')
df["datetime"] = pd.to_datetime(df["date"] + " "+ df["time"], dayfirst=True)

# df["datetime"]


In [8]:
df = df.set_index("datetime")
df

,date,time,sensor_id,value,activity,status
datetime,,,,,,
2009-02-02 07:15:17,02/02/2009,07:15:17,M35,ON,R1_Bed_to_Toilet,begin
2009-02-02 07:15:21,02/02/2009,07:15:21,AD1-A,2.82231,NaN,NaN
2009-02-02 07:15:23,02/02/2009,07:15:23,M35,OFF,NaN,NaN
2009-02-02 07:15:23,02/02/2009,07:15:23,M35,ON,NaN,NaN
2009-02-02 07:15:28,02/02/2009,07:15:28,AD1-A,2.79926,NaN,NaN
...,...,...,...,...,...,...
2009-04-04 07:26:16,04/04/2009,07:26:16,M48,OFF,NaN,NaN
2009-04-04 07:27:47,04/04/2009,07:27:47,M48,ON,NaN,NaN
2009-04-04 07:27:50,04/04/2009,07:27:50,M47,ON,NaN,NaN


In [9]:
# 检查时间都是前一个<=后一个
for a, b in zip(df.index, df.index[1:]):
    if(a>b):
        print(a)

# all(a <= b for a, b in zip(df.index, df.index[1:]))

## 1.4 给begin-end包围的activity打标签，begin-end之外的标为"other"，嵌套的按照最近的begin-end来标，标完之后删掉begin、end这一列。

In [9]:
# 将"value"列的"ON"和"OPEN"转为1，"OFF"和"CLOSE"转为0

# df["value"].unique() #检查value都有哪些名称

one_vals = ["ON","OPEN", "PRESENT"]
zero_vals = ["OFF","CLOSE", "ABSENT"]

df.loc[df['value'].isin(one_vals), 'value'] = 1
df.loc[df['value'].isin(zero_vals), 'value'] = 0
df


,date,time,sensor_id,value,activity,status
datetime,,,,,,
2009-02-02 07:15:17,02/02/2009,7:15:17,M35,1,R1_Bed_to_Toilet,begin
2009-02-02 07:15:21,02/02/2009,7:15:21,AD1-A,2.82231,NaN,NaN
2009-02-02 07:15:23,02/02/2009,7:15:23,M35,0,NaN,NaN
2009-02-02 07:15:23,02/02/2009,7:15:23,M35,1,NaN,NaN
2009-02-02 07:15:28,02/02/2009,7:15:28,AD1-A,2.79926,NaN,NaN
...,...,...,...,...,...,...
2009-04-04 07:26:16,04/04/2009,7:26:16,M48,0,NaN,NaN
2009-04-04 07:27:47,04/04/2009,7:27:47,M48,1,NaN,NaN
2009-04-04 07:27:50,04/04/2009,7:27:50,M47,1,NaN,NaN


In [10]:
# 检查begin总数==end总数
def check_begin_end_total(df):
    cnt_begin = 0
    cnt_end = 0
    for index,row in df.iterrows():
        if row.status == 'begin':
            cnt_begin += 1
        elif row.status == 'end':
            cnt_end += 1
    print(cnt_begin)
    print(cnt_end)
check_begin_end_total(df)

514
514


In [11]:
# 检查begin-end pair的activities可以相互抵消
def check_activity_symmetry(df):
    stack = []
    cnt = 0
    for index,row in df.iterrows():
        if row.activity != np.nan:
            if row.status == 'begin':
                stack.append(row.activity)
            elif row.status == 'end':
                stack.pop()

        # 每1000次打印一次栈的状态
        cnt += 1
        if cnt % 1000 == 0:
            print(stack)

    print("The final stack value:", stack)

check_activity_symmetry(df)

['Watch_TV']
['R1_Work', 'R2_Work']
['R1_Work']
['Meal_Preparation']
['Meal_Preparation']
['R1_Sleep', 'R2_Sleep']
['Meal_Preparation']
['R2_Work']
['R2_Work', 'R2_Sleep']
['Meal_Preparation']
['R1_Work']
['R1_Work', 'R2_Work']
['R2_Sleep', 'R1_Sleep']
['Meal_Preparation']
['R1_Work', 'Clean']
['Study', 'R1_Work']
['Study', 'R1_Work']
['R1_Sleep', 'R2_Sleep']
['Meal_Preparation']
['R1_Work']
['Watch_TV']
['Watch_TV', 'R2_Sleep']
['R1_Personal_Hygiene']
['Meal_Preparation']
['Meal_Preparation']
['Meal_Preparation']
[]
['Meal_Preparation']
['R1_Work', 'R2_Sleep']
['R1_Work', 'R1_Sleep', 'R1_Bed_to_Toilet']
['R1_Work', 'R2_Personal_Hygiene']
['Meal_Preparation', 'Watch_TV']
['Meal_Preparation', 'R1_Work']
['Meal_Preparation', 'R1_Work']
['Meal_Preparation', 'R1_Sleep']
['Watch_TV', 'Meal_Preparation']
['Watch_TV', 'Meal_Preparation']
['Watch_TV', 'R1_Work', 'R2_Work']
['R1_Sleep', 'R2_Sleep']
['Meal_Preparation']
['R1_Sleep', 'R2_Work']
['R1_Sleep', 'R1_Work']
['Meal_Preparation']
['Meal_

In [19]:
# 对activity填值：begin-end中间填最近的activity，begin-end外填other

# def set_activity(df):
stack = []
for index,row in df.iterrows():
    if row.activity != np.nan:
        if row.status == 'begin':
            stack.append(row.activity)
        elif row.status == 'end':
            stack.pop()

    else:
        # 判断有没有被begin-end包围
        if len(stack) == 0:
            df.at[index, "activity"] = "other"
        else:
            df.at[index, "activity"] = stack[-1]

    # return df

df

,date,time,sensor_id,value,activity,status
datetime,,,,,,
2009-02-02 07:15:17,02/02/2009,07:15:17,M35,ON,R1_Bed_to_Toilet,begin
2009-02-02 07:15:21,02/02/2009,07:15:21,AD1-A,2.82231,NaN,NaN
2009-02-02 07:15:23,02/02/2009,07:15:23,M35,OFF,NaN,NaN
2009-02-02 07:15:23,02/02/2009,07:15:23,M35,ON,NaN,NaN
2009-02-02 07:15:28,02/02/2009,07:15:28,AD1-A,2.79926,NaN,NaN
...,...,...,...,...,...,...
2009-04-04 07:26:16,04/04/2009,07:26:16,M48,OFF,NaN,NaN
2009-04-04 07:27:47,04/04/2009,07:27:47,M48,ON,NaN,NaN
2009-04-04 07:27:50,04/04/2009,07:27:50,M47,ON,NaN,NaN
